# Create a Twisted Interface between Nanoribbons

Use nanoribbon interface builder to create twisted interfaces between two nanoribbons.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create and visualize the initial nanoribbons
1. Generate twisted interface between nanoribbons
1. Visualize the final interface

## Notes
1. Nanoribbons are created from the input materials with specified width and length.
1. The top nanoribbon is rotated by the specified twist angle.
1. Vacuum is added along x and y directions to isolate the ribbons.
1. For more information, see [Introduction](Introduction.ipynb)


## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior


In [ ]:
# Material selection and basic parameters
FILM_INDEX = 0 # Index in the list of materials, to access as materials[FILM_INDEX]
SUBSTRATE_INDEX = None  # Can be None to use same material as film

# Twisted interface parameters
TARGET_TWIST_ANGLE = 17.9  # in degrees
INTERFACE_DISTANCE = 3.0  # in Angstroms
INTERFACE_VACUUM = 20.0  # in Angstroms

# Nanoribbon parameters
RIBBON_WIDTH = 3  # Width of the nanoribbon in unit cells
RIBBON_LENGTH = 3  # Length of the nanoribbon in unit cells
VACUUM_WIDTH = 15.0  # Vacuum width around ribbons in Angstroms
VACUUM_LENGTH = 15.0  # Vacuum length around ribbons in Angstroms
VACUUM_X = 2.0  # Additional vacuum along x on both sides, in Angstroms
VACUUM_Y = 2.0  # Additional vacuum along y on both sides, in Angstroms

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [3, 3, 1]

### 1.2. Install packages
Explanation is [here](under_the_hood.ipynb#1.2.-Install-packages).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("")

### 1.3. Load and preview input materials
Explanation is [here](under_the_hood.ipynb#2.-Data-Exchange).

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

 ## 2. Prepare Materials
 ### 2.1. Select and visualize initial materials

In [ ]:
from utils.visualize import visualize_materials

film = materials[FILM_INDEX]
substrate = materials[SUBSTRATE_INDEX] if SUBSTRATE_INDEX is not None else film

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(film, repetitions=VISUALIZE_REPETITIONS)
    if substrate is not film:
        visualize_materials(substrate, repetitions=VISUALIZE_REPETITIONS)

## 3. Generate Twisted Interface
### 3.1. Set up interface configuration and builder


In [ ]:
from mat3ra.made.tools.helpers import create_interface_twisted
# Create nanoribbons from the materials first
from mat3ra.made.tools.helpers import create_nanoribbon

# Create nanoribbons for both materials
nanoribbon1 = create_nanoribbon(
    material=film,
    width=RIBBON_WIDTH,  # Width in unit cells
    length=RIBBON_LENGTH,  # Length in unit cells
    vacuum_width=VACUUM_WIDTH,  # Vacuum width in Angstroms
    vacuum_length=VACUUM_LENGTH,  # Vacuum length in Angstroms
)

nanoribbon2 = create_nanoribbon(
    material=substrate,
    width=RIBBON_WIDTH,  # Width in unit cells
    length=RIBBON_LENGTH,  # Length in unit cells
    vacuum_width=VACUUM_WIDTH,  # Vacuum width in Angstroms
    vacuum_length=VACUUM_LENGTH,  # Vacuum length in Angstroms
)

# Create the twisted interface using the manual approach
interface = create_interface_twisted(
    material1=nanoribbon1,
    material2=nanoribbon2,
    angle=TARGET_TWIST_ANGLE,
    vacuum_x=VACUUM_X,
    vacuum_y=VACUUM_Y,
    gap=INTERFACE_DISTANCE,
)

### 3.2. Generate and analyze interfaces


In [ ]:
# The interface was already created above using the manual approach
print("\\nInterface created successfully!")
print(f"Target twist angle: {TARGET_TWIST_ANGLE}°")
print(f"Actual twist angle: {TARGET_TWIST_ANGLE}°")  # Since we set it manually
print(f"Number of atoms: {len(interface.basis.elements.ids)}")
print(f"Interface name: {interface.name}")

# Show intermediate visualization if requested
if SHOW_INTERMEDIATE_STEPS:
    from utils.visualize import visualize_materials
    print("\\nVisualizing the created interface:")
    visualize_materials(interface, repetitions=[1, 1, 1])
    visualize_materials(interface, repetitions=[1, 1, 1], rotation="-90x")

## 4. Preview the selected material
By default, the first interface is selected. You can change the selection by changing the `selected_interface` index.

### 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials
interface.name += f" ({TARGET_TWIST_ANGLE} degrees)"
set_materials(interface)